# Generating GANs with Desired Features

In [ ]:
# Run this for Google CoLab (use TensorFlow 1.x)
%tensorflow_version 1.x
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Next, clone StyleGAN2 from GitHub.

In [ ]:
!git clone https://github.com/NVlabs/stylegan2.git

Verify that StyleGAN has been cloned.

In [ ]:
!ls /content/stylegan2/

# Run StyleGAN2 From Python Code

In [ ]:
import sys
sys.path.insert(0, "/content/stylegan2")

import dnnlib

In [ ]:
# Copyright (c) 2019, NVIDIA Corporation. All rights reserved.
#
# This work is made available under the Nvidia Source Code License-NC.
# To view a copy of this license, visit
# https://nvlabs.github.io/stylegan2/license.html

import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys

import pretrained_networks

#----------------------------------------------------------------------------

def expand_seed(seeds, vector_size):
  result = []

  for seed in seeds:
    rnd = np.random.RandomState(seed)
    result.append( rnd.randn(1, vector_size) ) 
  return result

def generate_images(Gs, seeds, truncation_psi, prefix):
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if truncation_psi is not None:
        Gs_kwargs.truncation_psi = truncation_psi

    for seed_idx, seed in enumerate(seeds):
        print('Generating image for seed %d/%d ...' % (seed_idx, len(seeds)))
        rnd = np.random.RandomState(0)
        tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(seed, None, **Gs_kwargs) # [minibatch, height, width, channel]
        path = f"/content/{prefix}-{seed_idx+1}.png"
        PIL.Image.fromarray(images[0], 'RGB').save(path)


In [ ]:
sc = dnnlib.SubmitConfig()
sc.num_gpus = 1
sc.submit_target = dnnlib.SubmitTarget.LOCAL
sc.local.do_not_copy_source_files = True
sc.run_dir_root = "/content/drive/My Drive/projects/stylegan2"
sc.run_desc = 'generate-images'
network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'

print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)
vector_size = Gs.input_shape[1:][0]

# Explore Many GANs

Let's generate many different GAN's randomly.  Then find ones with the traits that we are looking for.

In [ ]:
import cv2 
from google.colab.patches import cv2_imshow

# 8227
for i in range(100):
  z = 9200+i
  print(z)
  seeds = expand_seed([z], vector_size)
  generate_images(Gs, seeds, 0.5, "image")
  img = cv2.imread('/content/image-1.png')   
  cv2_imshow(img) 

# What is an Image??

In [ ]:
sc = dnnlib.SubmitConfig()
sc.num_gpus = 1
sc.submit_target = dnnlib.SubmitTarget.LOCAL
sc.local.do_not_copy_source_files = True
sc.run_dir_root = "/content/drive/My Drive/projects/stylegan2"
sc.run_desc = 'generate-images'
network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'

print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)
vector_size = Gs.input_shape[1:][0]

In [ ]:
GLASSES = [2,3,17,19,26,40,42,50,44,65,68,70,85,90,96,112,
  130,139,140,146,147,152,155,164,176,183,194,198,209,214,215,241,244,245,254,260,
  267,296,300,312,313,322,324,331,333,346,361,366,386,401,412,422,425,443,448,472,
  487,494,501,516,544,545,551,552,565,568,591,594,595,597,607,610,639,640,669,670,
  675,682,703,710,719,730,757,764,765,767,777,780,792,798,799,802,808,812,819,823,
  837,857,866,870,881,887,893,894,895,899,915,916,934,936,941,973,979,988,991,993]

NO_GLASSES = [x for x in range(1000) if x not in GLASSES]

In [ ]:
import random
import pandas as pd

GEN_SET = GLASSES

def generate_set(GEN_SET, count):
  seeds3 = []

  for i in range(count):
    i1 = random.randint(0,len(GEN_SET)-1)
    i2 = random.randint(0,len(GEN_SET)-1)
    seeds1 = expand_seed([GEN_SET[i1]],vector_size)
    seeds2 = expand_seed([GEN_SET[i2]],vector_size)
    seeds = [seeds1[0][0], seeds2[0][0]]

    diff = seeds[1] - seeds[0]
    delta = (np.random.rand(512) * 0.6) + .2
    step = diff * delta
    current = [seeds[0].copy()]
    current += step
    current = np.round(current,5)
    seeds3.append(current)

  seeds4 = [x[0] for x in seeds3]
  seeds4 = np.array(seeds4)
  return pd.DataFrame(seeds4)



#generate_images(Gs, seeds3,0.5,'glasses')

df1 = generate_set(GLASSES, int(5000 * .64))
df1['glasses'] = 1
df2 = generate_set(NO_GLASSES, int(5000 * .36))
df2['glasses'] = 0

In [ ]:
df = pd.concat([df1,df2],ignore_index=True)

In [ ]:
v_cols2 = [f'v{i+1}' for i in range(512)]
cols2 = v_cols2.copy()
cols2.append('glasses')
df.columns = cols2
df = df.reindex(np.random.permutation(df.index))
df.insert(0, 'id', range(1,len(df)+1))

In [ ]:
l = len(df)
sz = int(l*.9)
df_train = df[0:sz].copy()
df_test = df[sz:].copy()
df_test.drop('glasses',axis=1,inplace=True)
df_solution = df[sz:].copy()
df_sample = df_test.copy()
df_sample['glasses'] = 0.64

In [ ]:
df_train.to_csv('/content/train.csv',index=False)
df_test.to_csv('/content/test.csv',index=False)
df_solution.to_csv('/content/solution.csv',index=False)
df_sample.to_csv('/content/sample.csv',index=False)


# Build Images

In [ ]:
df.shape

In [ ]:
seeds = df[v_cols2].values
seeds = [x.reshape([1,512]) for x in seeds]
generate_images(Gs, seeds,0.5,"face")